<link rel="preconnect" href="https://fonts.googleapis.com">
<link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
<link href="https://fonts.googleapis.com/css2?family=Titillium+Web:ital,wght@0,200;0,300;0,400;0,600;0,700;0,900;1,200;1,300;1,400;1,600;1,700&display=swap" rel="stylesheet">
<div class="box" style="padding: 10px; margin: 100px 90px; background-color: white; color: white; border-radius: 5px; font-size: 15px; box-shadow: rgba(0, 0, 0, 0.25) 0px 54px 55px, rgba(0, 0, 0, 0.12) 0px -12px 30px, rgba(0, 0, 0, 0.12) 0px 4px 6px, rgba(0, 0, 0, 0.17) 0px 12px 13px, rgba(0, 0, 0, 0.09) 0px -3px 5px;">
  <table style="padding: 10px; margin: auto auto;   border-radius: 20px; font-size: 15px;">
      <tr>
          <th colspan='2'><h1 style="text-align: center">
Natural Language Processing</br>
</h1>
<h2 style="text-align: center">
Course Assignment Two </br>
</h2>
</th>
<tr>
    <tr>
      <th colspan="2">Personal Info</th>
    </tr>
    <tr>
      <td>Ali</td>
    <td><b>Nikkhah</b></td>
    </tr>
    <tr>
      <td>Sarina</td>
    <td><b>Zahedi</b></td>
    </tr>
    <tr>
      <td>Ramtin</td>
    <td><b>Khoshnevis</b></td>
    </tr>
    <tr>
      <td>Github:</td>
      <td><a href="https://github.com/AliNikkhah2001/NaturalLanguageProcessing02" target="_blank">https://github.com/AliNikkhah2001/NaturalLanguageProcessing02</a></td>
    </tr>
  </table>
</div>


## Initializations

In [1]:
%%capture
!pip install transformers datasets torch
!pip install pandas tqdm
!pip install python-Levenshtein
!pip install jellyfish


In [58]:
# Import necessary libraries from Hugging Face and PyTorch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset, Dataset
from tqdm.notebook import tqdm 
import random
import torch
import pandas as pd
import os
import re
import Levenshtein
import pandas as pd

In [59]:
!nvidia-smi

Thu May 30 04:23:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0              35W / 250W |  10558MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Load of the Datasets

In [60]:
datasetPath="/kaggle/input/nlpsphw2"


### Error in words dataset

In [61]:

# Define the paths to the datasets
faspell_main_path = datasetPath + '/faspell_main.txt'
faspell_ocr_path = datasetPath + '/faspell_ocr.txt'

# Read and clean the faspell_main dataset
faspell_main_df = pd.read_csv(faspell_main_path, sep='\t', encoding='utf-8', header=None, names=["misspelled", "corrected", "error-category"])
faspell_main_cleaned_df = faspell_main_df.rename(columns={"corrected": "wrong", "misspelled": "correct"})
faspell_main_cleaned_df = faspell_main_cleaned_df[['correct', 'wrong']]

# Read and clean the faspell_ocr dataset
faspell_ocr_df = pd.read_csv(faspell_ocr_path, sep='\t', encoding='utf-8', header=None, names=["misspelled", "corrected"])
faspell_ocr_cleaned_df = faspell_ocr_df.rename(columns={"corrected": "wrong", "misspelled": "correct"})
faspell_ocr_cleaned_df = faspell_ocr_cleaned_df[['correct', 'wrong']]

# Concatenate the cleaned dataframes
words_df = pd.concat([faspell_main_cleaned_df, faspell_ocr_cleaned_df], ignore_index=True)

# Print samples from the final DataFrame
print(words_df.sample(5))

      correct     wrong
4055  نپیچیند    نپیچید
5100      حوب       خوب
3692  ميآوريم  مي آوريم
1499    خارحي     خارجي
3296     محمع      مجمع


### Synthetic dataset generation of typo

In [62]:
# Load the list of Persian words
distinct_words_path = '/kaggle/input/distinctwords/distinct_words.txt'
with open(distinct_words_path, 'r', encoding='utf-8') as file:
    words = file.read().splitlines()

# Functions to create typographical errors
def missing_char(word):
    pos = random.randint(0, len(word) - 1)
    return word[:pos] + word[pos + 1:]

def extra_char(word):
    pos = random.randint(0, len(word))
    char = random.choice('ابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی')
    return word[:pos] + char + word[pos:]

def swapped_chars(word):
    if len(word) < 5:
        return word
    pos1, pos2 = random.sample(range(len(word)), 2)
    word_list = list(word)
    word_list[pos1], word_list[pos2] = word_list[pos2], word_list[pos1]
    return ''.join(word_list)




In [63]:
additional_data = []
for word in tqdm(words, desc="Generating synthetic data"):
    if(len(word)>5):
        for _ in range(20):
            wordtype = random.choice([missing_char, extra_char, swapped_chars])
            gen_word = wordtype(word)
            additional_data.append({'correct': gen_word, 'wrong': word})

# Create a DataFrame from the additional data
additional_df = pd.DataFrame(additional_data)

# Append the additional data to the original DataFrame
words_df = pd.concat([words_df, additional_df], ignore_index=True)



Generating synthetic data:   0%|          | 0/453158 [00:00<?, ?it/s]

## ADD HE-KASRE TO DF

In [64]:
# Load the annotated dataset
annotated_path = '/kaggle/input/nlpsphw2/annotated.csv'
annotated_df = pd.read_csv(annotated_path, encoding='utf-8')

# Extract the name of the single column
column_name = annotated_df.columns[0]

# Function to process each row
def process_text(row):
    text = row[column_name]
    words = text.split()  # Split the text into words
    correct_words = []
    wrong_words = []
    
    for word in words:
        if word.endswith('ه+'):
            correct_word = word[:-2]  # Remove 'ه+'
            wrong_word = word.replace('+', '')  # Remove only the '+'
            correct_words.append(correct_word)
            wrong_words.append(wrong_word)
    
    correct_text = ' '.join(correct_words)
    wrong_text = ' '.join(wrong_words)
    
    return pd.Series([correct_text, wrong_text], index=['correct', 'wrong'])

# Apply the process_text function to each row in the DataFrame
tmp_df = annotated_df.apply(process_text, axis=1)
words_df = pd.concat([words_df, tmp_df], ignore_index=True)


In [65]:
# Print samples from the final DataFrame
print(words_df.sample(20))

               correct         wrong
6222240        تونشکین        تونکین
6054334  نمازجمعذه‌است  نمازجمعه‌است
1418416        رنجیابی      رنج‌یابی
4104848     ناامیدششدن     ناامیدشدن
499408        خان‌کنید      خان‌کندی
1964587         مافورد       مامفورد
2197184          کج‌قل        کج‌قلم
1798982      روضه‌خانى    روضه‌خوانى
4211377      هیپوتیزور    هیپنوتیزور
3107284    کامبوجچی‌ها    کامبوجی‌ها
5119134         فریازن       فریازان
5426767        الناشدد        الناشد
5547876   پایفبندی‌های   پایبندی‌های
1969865        کالجدار        کالدار
3802962       هاده‌اید     نهاده‌اید
2171856       کردازینت       کردزینت
4059989          بدعلی        بدعملی
4431442     موزی‌ویدئو   موزیک‌ویدئو
5019579       قایلیتاب      قابلیتای
722993      رشته‌گرنفی    رشته‌فرنگی


In [66]:
dataset_size = words_df.size
print(f"Size of the dataset: {dataset_size}")

# Print number of columns
num_columns = words_df.shape[1]
print(f"Number of columns: {num_columns}")

# Print number of rows
num_rows = words_df.shape[0]
print(f"Number of rows: {num_rows}")




Size of the dataset: 13296918
Number of columns: 2
Number of rows: 6648459


In [67]:
# Write the DataFrame to a tab-separated file
words_df_path = '/kaggle/working/created_words_dataset.txt'
words_df.to_csv(words_df_path, sep='\t', index=False)

print(f"DataFrame written to {words_df_path}")

DataFrame written to /kaggle/working/created_words_dataset.txt


## Sentences Dataset

In [68]:

real_word_path = '/kaggle/input/nlpsphw2/DataSet/DataSet/real-word'
real_word_files = ['gozar', 'plural', 'be', 'tanvin', 'hich', 'common']
real_word_files

['gozar', 'plural', 'be', 'tanvin', 'hich', 'common']

In [69]:

data = []

for file_name in real_word_files:
    correct_file = os.path.join(real_word_path, f'{file_name}/correct_{file_name}.txt')
    wrong_file = os.path.join(real_word_path, f'{file_name}/wrong_{file_name}.txt')
    
    with open(correct_file, 'r', encoding='utf-8') as f:
        correct_lines = f.readlines()
        
    with open(wrong_file, 'r', encoding='utf-8') as f:
        wrong_lines = f.readlines()
    
    for correct, wrong in zip(correct_lines, wrong_lines):
        data.append({'correct': correct.strip(), 'wrong': wrong.strip()})

sentences_df = pd.DataFrame(data)



In [70]:
# Load the dataset
dehkhoda_corpus = pd.read_csv('/kaggle/input/dehkhoda/dehkhoda_corpus.csv')

# Assuming the dataset has columns 'wrong' and 'correct'
wrong = dehkhoda_corpus['wrong']
correct = dehkhoda_corpus['correct']

# Convert to DataFrame if needed
wrong_df = wrong.to_frame(name='wrong')
correct_df = correct.to_frame(name='correct')

# Combine the wrong and correct DataFrames into one
combined_df = pd.concat([wrong_df, correct_df], axis=1)

# Append to sentences_df
sentences_df = pd.concat([sentences_df, combined_df], ignore_index=True)
# Display the updated sentences_df
print(sentences_df.head())

                                             correct  \
0  اما گروه دوم افزایش تدریجی قابل پیش‌بینی و مشر...   
1  دانشگاه‌ها حوزه اثرگذاری چه قبل از انقلاب و چه...   
2         به‌اندازه ظرفیت خودش بر بازار اثرگذار باشد   
3  که تشکیل دادسرای امور بین‌الملل برای رسیدگی به...   
4  در صورت نیاز نسبت به اصلاح نسبت به اثرگذاری مت...   

                                               wrong  
0  اما گروه دوم افزایش تدریجی قابل پیش‌بینی و مشر...  
1  دانشگاه‌ها حوزه اثر‌گزاری چه قبل از انقلاب و چ...  
2        به‌اندازه ظرفیت خودش بر بازار اثر‌گزار باشد  
3  که تشکیل دادسرای امور بین‌الملل برای رسیدگی به...  
4  در صورت نیاز نسبت به اصلاح نسبت به اثر‌گزاری م...  


In [71]:
# Convert to DataFrame
synthetic_df = pd.DataFrame(data)
print(synthetic_df.sample(5))
dataset_size = synthetic_df.size
print(f"Size of the dataset: {dataset_size}")

# Print number of columns
num_columns = synthetic_df.shape[1]
print(f"Number of columns: {num_columns}")

# Print number of rows
num_rows = synthetic_df.shape[0]
print(f"Number of rows: {num_rows}")




                                                correct  \
1712      قانونگذار به این دلیل اختیار را به دولت نسپرد   
2467  همین شکل و شمایل و آداب و هنر و رسوءم و پوشاک ...   
39                              ما باج‌گزار شما می‌شویم   
4976  و درباره موضعات مهمی از جمله چگونگی بهبود و ار...   
6209    اگر سهامداران شرکت‌ها را متعلق به خودشان بدانند   

                                                  wrong  
1712       قانوگذار به این دلیل اختیار را به دولت نسپرد  
2467  همین شکل و شمایل و آداب‌ها و هنر و رسوم و پوشا...  
39                              ما باج‌گذار شما می‌شویم  
4976  و درباره موضعات مهمی از جمله چگونگی بهبود و ار...  
6209      اگر سهامداران شرکت‌ها را متعلق بخودشان بدانند  
Size of the dataset: 15932
Number of columns: 2
Number of rows: 7966


In [72]:
sentences_df = pd.concat([sentences_df, synthetic_df], ignore_index=True)


In [73]:
dataset_size = sentences_df.size
print(f"Size of the dataset: {dataset_size}")

# Print number of columns
num_columns = sentences_df.shape[1]
print(f"Number of columns: {num_columns}")

# Print number of rows
num_rows = sentences_df.shape[0]
print(f"Number of rows: {num_rows}")


words_df_path = 'created_sentences_dataset.txt'
words_df.to_csv(words_df_path, sep='\t', index=False)

print(f"DataFrame written to {words_df_path}")

Size of the dataset: 33280
Number of columns: 2
Number of rows: 16640
DataFrame written to created_sentences_dataset.txt


## Dataset for Hekasre

In [74]:
annotated_path = datasetPath+'/annotated.csv'
annotated_df = pd.read_csv(annotated_path, encoding='utf-8')
annotated_df = pd.DataFrame(annotated_df)
# Extract the name of the single column
column_name = annotated_df.columns[0]

# Function to process each row
def process_text(row):
    text = row[column_name]
    correct_text = text.replace('ه+', '')
    wrong_text = text.replace('+', '')
    return pd.Series([correct_text, wrong_text], index=['correct', 'wrong'])

# Apply the function to each row and create a new DataFrame
result_df = annotated_df.apply(process_text, axis=1)
print(result_df.sample(5))
dataset_size = result_df.size
print(f"Size of the dataset: {dataset_size}")

# Print number of columns
num_columns = result_df.shape[1]
print(f"Number of columns: {num_columns}")

# Print number of rows
num_rows = result_df.shape[0]
print(f"Number of rows: {num_rows}")



                                               correct  \
515  بچه هفت سال دارد پدرش قصاب است با مادرش اختلاف...   
723  حسین فریدون برادر رییس جمهور به اتهام دریافت ر...   
124  اتهامات فریدون تاسیس صرافی دست داشتن در انتصاب...   
549  بیژن قاسم زاده بازپرس سابق شعبه دوم دادسرای فر...   
238  وقتش- که رهبر اغتشاشات فرانسه رو بیاریم قم و د...   

                                                 wrong  
515  بچه هفت سال دارد پدرش قصاب است با مادرش اختلاف...  
723  حسین فریدون برادره رییس جمهور به اتهامه دریافت...  
124  اتهاماته فریدون تاسیسه صرافی دست داشتن در انتص...  
549  بیژنه قاسم زاده بازپرسه سابقه شعبه دوم دادسرای...  
238  وقتش- که رهبر اغتشاشاته فرانسه رو بیاریم قم و ...  
Size of the dataset: 1998
Number of columns: 2
Number of rows: 999


In [75]:
sentences_df = pd.concat([sentences_df, result_df], ignore_index=True)


In [76]:
dataset_size = sentences_df.size
print(f"Size of the dataset: {dataset_size}")

# Print number of columns
num_columns = sentences_df.shape[1]
print(f"Number of columns: {num_columns}")

# Print number of rows
num_rows = sentences_df.shape[0]
print(f"Number of rows: {num_rows}")


words_df_path = 'created_sentences_dataset.txt'
words_df.to_csv(words_df_path, sep='\t', index=False)

print(f"DataFrame written to {words_df_path}")

Size of the dataset: 35278
Number of columns: 2
Number of rows: 17639
DataFrame written to created_sentences_dataset.txt


# Distance Model

In [77]:
import jellyfish

In [ ]:
import Levenshtein
import jellyfish
import pandas as pd

class DistanceModel:
    def __init__(self, words_path, words_df_path=None):
        """
        Initializes the DistanceModel with a list of Persian words and a correction DataFrame.
        
        Args:
        words_path (str): Path to the file containing Persian words.
        words_df_path (str): Path to the file containing the correction DataFrame.
        """
        self.words = self.load_words(words_path)
        self.words_set = set(self.words)  # Use a set for O(1) average-time complexity lookups
        self.soundex_dict = self.build_soundex_dict(self.words)

    def load_words(self, path):
        """
        Loads words from a specified file.
        
        Args:
        path (str): Path to the file containing Persian words.
        
        Returns:
        list: List of words.
        """
        with open(path, 'r', encoding='utf-8') as file:
            words = file.read().splitlines()
        return words

    def load_words_df(self, path):
        """
        Loads the correction DataFrame from a specified file.
        
        Args:
        path (str): Path to the file containing the correction DataFrame.
        
        Returns:
        DataFrame: DataFrame containing 'wrong' and 'correct' columns.
        """
        return pd.read_csv(path, sep='\t')  # Assuming the file is tab-separated

    def build_soundex_dict(self, words):
        """
        Builds a dictionary mapping Soundex codes to lists of words.
        
        Args:
        words (list): List of words to build the Soundex dictionary.
        
        Returns:
        dict: Dictionary mapping Soundex codes to lists of words.
        """
        soundex_dict = {}
        for word in words:
            soundex_code = jellyfish.soundex(word)
            if soundex_code not in soundex_dict:
                soundex_dict[soundex_code] = []
            soundex_dict[soundex_code].append(word)
        return soundex_dict

    def calculate_distance(self, word1, word2):
        """
        Calculates the Levenshtein distance between two words.
        
        Args:
        word1 (str): The first word.
        word2 (str): The second word.
        
        Returns:
        int: The Levenshtein distance between the two words.
        """
        return Levenshtein.distance(word1, word2)

    def find_closest_word(self, word):
        """
        Finds the closest word in the loaded list of Persian words to the input word.
        
        Args:
        word (str): The input word to find the closest match for.
        
        Returns:
        str: The closest word from the list of Persian words.
        """
        soundex_code = jellyfish.soundex(word)
        candidate_words = self.soundex_dict.get(soundex_code, self.words)
        
        min_distance = float('inf')
        closest_word = None
        
        for persian_word in candidate_words:
            distance = self.calculate_distance(word, persian_word)
            if distance < min_distance:
                min_distance = distance
                closest_word = persian_word
        
        return closest_word

    def correct_words(self, words):
        """
        Corrects words in the input list using the words_df DataFrame.
        
        Args:
        words (list): List of words to be corrected.
        
        Returns:
        list: List of corrected words.
        """
        wrong_to_correct = dict(zip(self.words_df['wrong'], self.words_df['correct']))
        corrected_words = [wrong_to_correct.get(word, word) for word in words]
        return corrected_words

    def inference(self, sentence):
        """
        Processes the input sentence and returns a cleaned version with words replaced 
        by their closest matches from the list of Persian words.
        
        Args:
        sentence (str): The input sentence to be cleaned.
        
        Returns:
        str: The cleaned sentence with words replaced by their closest matches.
        """
        # Split the sentence into words
        words = sentence.split()
        
        # Correct the words using the words_df DataFrame
        #words = self.correct_words(words)
        
        # Replace each word with its closest match from the list of Persian words
        cleaned_sentence = ' '.join(word if word in self.words_set else self.find_closest_word(word) for word in words)
        
        return cleaned_sentence



In [79]:
!ls

created_sentences_dataset.txt  created_words_dataset.txt  results


In [80]:
# Example usage:
words_path = '/kaggle/input/persianwords/big.txt'
words_df_path = '/kaggle/working/created_words_dataset.txt'  

model = DistanceModel(words_path, words_df_path)


In [81]:
# Sample sentence for inference
sample_sentence = "منه بهه خیلیی میرویدی"
cleaned_sentence = model.inference(sample_sentence)
print(cleaned_sentence)
print()

منه بهره خیلی میروید



In [82]:
input_sentence = input("Enter a sentence: ")
cleaned_sentence = model.inference(sample_sentence)
print("Corrected Sentence:", input_sentence)


Enter a sentence:  


Corrected Sentence: 


In [83]:
sample_sentences = [
    "هن به مدسه رفتم",
    "امروز هوا خیلیی خوب است",
    "من کتاب می‌خوامنم",
    "دیروز با دوستمم به پارک رفتم",
    "من عاقش یادگری هسنم"
]

for sentence in sample_sentences:
    corrected_sentence = model.inference(sentence)
    print(f"Original: {sentence}")
    print(f"Corrected: {corrected_sentence}\n")


Original: هن به مدسه رفتم
Corrected: هن به مدسه رفتم

Original: امروز هوا خیلیی خوب است
Corrected: امروز هوا خیلی خوب است

Original: من کتاب می‌خوامنم
Corrected: من کتاب می‌خوانم

Original: دیروز با دوستمم به پارک رفتم
Corrected: دیروز با دوستم به پارک رفتم

Original: من عاقش یادگری هسنم
Corrected: من عاقل یادگیری هستم



# T5 Model Training


In [84]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from torch.utils.data import DataLoader
from accelerate import Accelerator, DataLoaderConfiguration
import os


# Disable wandb logging
os.environ["WANDB_DISABLED"] = "true"

# Model and tokenizer
model_name = "Ahmad/parsT5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [85]:

# Adding special tokens to the tokenizer
special_tokens_dict = {'additional_special_tokens': ['<extra_token_1>', '<extra_token_2>']}
tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))


Embedding(32105, 768)

In [86]:
words_df = pd.DataFrame(words_df)


In [87]:
from torch.utils.data import Dataset

class WordCorrectionDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        input_text = self.dataframe.iloc[idx]["wrong"]
        target_text = self.dataframe.iloc[idx]["correct"]
        input_encoding = self.tokenizer(
            input_text, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt"
        )
        target_encoding = self.tokenizer(
            target_text, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt"
        )
        labels = target_encoding["input_ids"]
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            "input_ids": input_encoding["input_ids"].flatten(),
            "attention_mask": input_encoding["attention_mask"].flatten(),
            "labels": labels.flatten(),
        }

    def print_samples(self, num_samples=5):
        """Prints a specified number of samples from the dataset."""
        print(f"{'Index':<10}{'Wrong Word':<20}{'Correct Word'}")
        print("="*50)
        for idx in range(min(num_samples, len(self.dataframe))):
            wrong = self.dataframe.iloc[idx]["wrong"]
            correct = self.dataframe.iloc[idx]["correct"]
            print(f"{idx:<10}{wrong:<20}{correct}")

class SentenceCorrectionDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        input_text = self.dataframe.iloc[idx]["wrong"]
        target_text = self.dataframe.iloc[idx]["correct"]
        input_encoding = self.tokenizer(
            input_text, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt"
        )
        target_encoding = self.tokenizer(
            target_text, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt"
        )
        labels = target_encoding["input_ids"]
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            "input_ids": input_encoding["input_ids"].flatten(),
            "attention_mask": input_encoding["attention_mask"].flatten(),
            "labels": labels.flatten(),
        }

    def print_samples(self, num_samples=5):
        """Prints a specified number of samples from the dataset."""
        print(f"{'Index':<10}{'Wrong Sentence':<50}{'Correct Sentence'}")
        print("="*100)
        for idx in range(min(num_samples, len(self.dataframe))):
            wrong = self.dataframe.iloc[idx]["wrong"]
            correct = self.dataframe.iloc[idx]["correct"]
            print(f"{idx:<10}{wrong:<50}{correct}")


In [88]:
# Adding special tokens to the tokenizer
special_tokens_dict = {'additional_special_tokens': ['<extra_token_1>', '<extra_token_2>']}
tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

# Initialize the word-level dataset
word_dataset = WordCorrectionDataset(words_df, tokenizer)
word_dataset.print_samples(num_samples=5)

Index     Wrong Word          Correct Word
0         corrected           #misspelt
1         آگاهي               آاهي
2         آیات                آبات
3         آب باشد             آبباشد
4         آید                 آبد


In [91]:
sentence_dataset = SentenceCorrectionDataset(sentences_df, tokenizer)
sentence_dataset.print_samples(num_samples=5)

Index     Wrong Sentence                                    Correct Sentence
0         اما گروه دوم افزایش تدریجی قابل پیش‌بینی و مشروط را دارای اثر‌گزاری بیشتری می‌داننداما گروه دوم افزایش تدریجی قابل پیش‌بینی و مشروط را دارای اثرگذاری بیشتری می‌دانند
1         دانشگاه‌ها حوزه اثر‌گزاری چه قبل از انقلاب و چه بعد از انقلاب بوده‌انددانشگاه‌ها حوزه اثرگذاری چه قبل از انقلاب و چه بعد از انقلاب بوده‌اند
2         به‌اندازه ظرفیت خودش بر بازار اثر‌گزار باشد       به‌اندازه ظرفیت خودش بر بازار اثرگذار باشد
3         که تشکیل دادسرای امور بین‌الملل برای رسیدگی به این مهم اثر‌گزار باشدکه تشکیل دادسرای امور بین‌الملل برای رسیدگی به این مهم اثرگذار باشد
4         در صورت نیاز نسبت به اصلاح نسبت به اثر‌گزاری متن اقدام تا پس از اصلاح برای امضا تقدیم حضور گردددر صورت نیاز نسبت به اصلاح نسبت به اثرگذاری متن اقدام تا پس از اصلاح برای امضا تقدیم حضور گردد


In [89]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=200,
    per_device_train_batch_size=512,
    per_device_eval_batch_size=512,
    gradient_accumulation_steps=2,  # Accumulate gradients over 2 steps
    save_steps=10_000,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_steps=500,
    report_to="none",
    fp16=True,  # Enable mixed precision training
    dataloader_num_workers=4,
)

In [90]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=word_dataset,
    eval_dataset=word_dataset,  # Ideally, use a separate validation set
    tokenizer=tokenizer,
)

# Prepare dataloaders with Accelerator
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
accelerator = Accelerator(dataloader_config=dataloader_config)

train_loader = DataLoader(word_dataset, batch_size=training_args.per_device_train_batch_size, shuffle=True)
eval_loader = DataLoader(word_dataset, batch_size=training_args.per_device_eval_batch_size)

train_loader, eval_loader, model = accelerator.prepare(
    train_loader, eval_loader, model
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [57]:
# Disable parallelism in tokenizers to avoid deadlocks
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# Train the model on sentence-level dataset
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=200, training_loss=0.0, metrics={'train_runtime': 220.7309, 'train_samples_per_second': 3.624, 'train_steps_per_second': 0.906, 'total_flos': 547762470912000.0, 'train_loss': 0.0, 'epoch': 200.0})

In [96]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=2,  # Reduced batch size
    per_device_eval_batch_size=2,   # Reduced batch size
    gradient_accumulation_steps=4,  # Increase gradient accumulation steps
    save_steps=10_000,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_steps=500,
    report_to="none",
    fp16=True,  # Enable mixed precision training
    dataloader_num_workers=4,
)

In [97]:

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=sentence_dataset,
    eval_dataset=sentence_dataset,  # Ideally, use a separate validation set
    tokenizer=tokenizer,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [98]:
# Prepare dataloaders with Accelerator
accelerator = Accelerator()
train_loader = DataLoader(sentence_dataset, batch_size=training_args.per_device_train_batch_size, shuffle=True)
eval_loader = DataLoader(sentence_dataset, batch_size=training_args.per_device_eval_batch_size)

model, train_loader, eval_loader = accelerator.prepare(
    model, train_loader, eval_loader
)

In [ ]:
# Train the model on sentence-level dataset
trainer.train()

Step,Training Loss,Validation Loss


In [ ]:
# Save the model
model_save_path = "./fine_tuned_t5_model"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

## GPT2-PERSIAN

In [ ]:
# Load GPT-2 model and tokenizer
model_name = "HooshvareLab/gpt2-fa"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=2,  # Reduced batch size
    per_device_eval_batch_size=2,   # Reduced batch size
    gradient_accumulation_steps=4,  # Increase gradient accumulation steps
    save_steps=10_000,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_steps=500,
    report_to="none",
    fp16=True,  # Enable mixed precision training
    dataloader_num_workers=4,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=sentence_dataset,
    eval_dataset=sentence_dataset,  # Ideally, use a separate validation set
    tokenizer=tokenizer,
)
# Prepare dataloaders with Accelerator
accelerator = Accelerator()
train_loader = DataLoader(sentence_dataset, batch_size=training_args.per_device_train_batch_size, shuffle=True)
eval_loader = DataLoader(sentence_dataset, batch_size=training_args.per_device_eval_batch_size)

model, train_loader, eval_loader = accelerator.prepare(
    model, train_loader, eval_loader
)

# Print a sample output before training
def print_sample_output(model, tokenizer, sample_text):
    model.eval()
    inputs = tokenizer(sample_text, return_tensors="pt").to(accelerator.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=512, num_beams=5, early_stopping=True)
    corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Sample input: {sample_text}")
    print(f"Model output: {corrected_text}")

sample_text = sentences_df.iloc[0]["wrong"]
print_sample_output(model, tokenizer, sample_text)


In [ ]:
trainer.train()

## Final Model and i-o T5

In [ ]:
from transformers import T5ForConditionalGeneration, AutoTokenizer

class CombinedModel:
    def __init__(self, model_path, words_path, words_df_path):
        """
        Initializes the CombinedModel with the fine-tuned T5 model and the DistanceModel.
        
        Args:
        model_path (str): Path to the directory containing the fine-tuned T5 model.
        words_path (str): Path to the file containing Persian words.
        words_df_path (str): Path to the file containing the correction DataFrame.
        """
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = T5ForConditionalGeneration.from_pretrained(model_path)
        self.distance_model = DistanceModel(words_path, words_df_path)

    def inference(self, sentence):
        """
        Processes the input sentence using both the T5 model and the DistanceModel.
        
        Args:
        sentence (str): The input sentence to be processed.
        
        Returns:
        str: The processed sentence.
        """
        # Step 1: Use the DistanceModel to clean the sentence
        cleaned_sentence = self.distance_model.inference(sentence)
        
        # Step 2: Use the T5 model to generate the final output
        input_ids = self.tokenizer.encode(cleaned_sentence, return_tensors='pt')
        with torch.no_grad():
            outputs = self.model.generate(input_ids, max_length=512, num_beams=5, early_stopping=True)
        final_output = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        input_words = sentence.split()
        if len(final_output_words) != len(input_words):
            final_output_words = final_output_words[:len(input_words)]  # Truncate if longer
            if len(final_output_words) < len(input_words):
                final_output_words += input_words[len(final_output_words):]  # Pad if shorter
        
        return ' '.join(final_output_words)



In [ ]:
# Example usage
model_path = "./fine_tuned_t5_model"
words_path = "path_to_words_file.txt"
words_df_path = "path_to_words_df.csv"

combined_model = CombinedModel(model_path, words_path, words_df_path)



# Final Model and i-o GPT

In [ ]:

class CombinedModel:
    def __init__(self, model_path, words_path, words_df_path):
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModelForCausalLM.from_pretrained(model_path)
        self.distance_model = DistanceModel(words_path, words_df_path)

    def inference(self, sentence):
        cleaned_sentence = self.distance_model.inference(sentence)
        input_ids = self.tokenizer.encode(cleaned_sentence, return_tensors='pt')
        with torch.no_grad():
            outputs = self.model.generate(input_ids, max_length=512, num_beams=5, early_stopping=True)
        final_output = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        final_output_words = final_output.split()
        input_words = sentence.split()
        if len(final_output_words) != len(input_words):
            final_output_words = final_output_words[:len(input_words)]
            if len(final_output_words) < len(input_words):
                final_output_words += input_words[len(final_output_words):]
        
        return ' '.join(final_output_words)

# Example usage
model_path = "./fine_tuned_gpt2_model"
words_path = "path_to_words_file.txt"
words_df_path = "path_to_words_df.csv"

combined_model = CombinedModel(model_path, words_path, words_df_path)